* Data from https://opendata.dwd.de/climate_environment/GPCC/html/fulldata-monthly_v2022_doi_download.html

In [2]:
using YAXArrays,NetCDF

In [24]:
dataset=open_dataset("full_data_monthly_v2022_2011_2020_25.nc",driver=:netcdf)["precip"]

144×72×120 YAXArray{Union{Missing, Float32},3} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} 88.75:-2.5:-88.75 ReverseOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2011-01-01T00:00:00, …, 2020-12-01T00:00:00] ForwardOrdered Irregular Points
units: mm/month
name: precip
Total size: 4.75 MB


In [74]:
using Statistics

In [125]:
max_val=mapslices(maximum∘skipmissing,dataset;dims="Ti")
mean_val=mapslices(mean∘skipmissing,dataset;dims="Ti")
std_val=mapslices(std∘skipmissing,dataset;dims="Ti")

144×72 YAXArray{Union{Missing, Float32},2} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} 88.75:-2.5:-88.75 ReverseOrdered Regular Points
Total size: 40.5 KB


In [34]:
using GLMakie, GeoMakie

In [55]:
using DimensionalData

In [60]:
lats=lookup(max_val,:lat)

Sampled{Float64} ReverseOrdered Regular Points
wrapping: 88.75:-2.5:-88.75

In [61]:
lons=lookup(max_val,:lon)

Sampled{Float64} ForwardOrdered Regular Points
wrapping: -178.75:2.5:178.75

In [93]:
using GeoJSON

In [94]:
worldCountries = GeoJSON.read("../ne_110m_admin_0_countries.geojson")

FeatureCollection with 177 Features

In [108]:
max_val.data[:,:]

144×72 Matrix{Union{Missing, Float32}}:
 missing  missing  missing  missing  …  missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing  …  missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  missing
 ⋮                                   ⋱                    ⋮        
 missing  missing  missing  missing  …  missing  missing  missing  missing
 missing  missing  missing  missing     missing  missing  missing  

In [155]:
findfirst(lons.>20)


81

In [118]:
findfirst(lons.>170)

141

In [114]:
findfirst(lats.<-10)

41

In [115]:
lats[:]

Sampled{Float64} ReverseOrdered Regular Points
wrapping: 88.75:-2.5:-88.75

In [1]:
fig=Figure(resolution=(800,1000))
Label(fig[1,1:2],text="Rain in Asia a wide spread from little to a lot\n based on GPCC Full-Data monthly 2.5° (2011-2020)",fontsize=30)
ax=GeoAxis(fig[2,1],dest="+proj=eqearth +lon_0=150 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs",lonlims=(20,180),latlims=(-10,90),title="Mean")
mean_s=surface!(ax,lons[81:end],lats[1:41],mean_val[81:end,1:41],colorrange=(0,200),shading=false,colormap=cgrad(:deep,rev=false),highclip=:black)
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :white,
    strokewidth = 0.25)
translate!(hm1, 0, 0, 1000)
Colorbar(fig[2,2],vertical=true,mean_s,label="Mean mm/month")


ax=GeoAxis(fig[3,1],dest="+proj=eqearth +lon_0=150 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs",lonlims=(20,180),latlims=(-10,90),title="Maximum")
max_s=surface!(ax,lons[81:end],lats[1:41],max_val[81:end, 1:41],colorrange=(0,1000),shading=false,colormap=cgrad(:deep,rev=false),highclip=:black)
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :white,
    strokewidth = 0.25)
translate!(hm1, 0, 0, 1000)
Colorbar(fig[3,2],vertical=true,max_s,label="Maximum mm/month")

ax=GeoAxis(fig[4,1],dest="+proj=eqearth +lon_0=150 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs",lonlims=(20,180),latlims=(-10,90),title="Standard deviation")
std_s=surface!(ax,lons[81:end],lats[1:41],std_val[81:end, 1:41],colorrange=(0,100),shading=false,colormap=cgrad(:fire,rev=false),highclip=:white)
Colorbar(fig[4,2],vertical=true,std_s,label="Std mm/month")
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :white,
    strokewidth = 0.25)
translate!(hm1, 0, 0, 1000)
Label(fig[5,1:2],text="Data: © Deutscher Wetterdienst\nMade with Natural Earth",justification=:left)
Makie.resize_to_layout!(fig)
fig

UndefVarError: UndefVarError: `Figure` not defined

In [208]:
save("asia_noshading.png",fig)

In [198]:
Makie.spaces()

(:data, :pixel, :relative, :clip)